Set up all columns before any fetching is done
set up 'root's (rankings, server file)
For every iteration, it should fetch all unknowns, store their "wave", then extract all unseen players from that cohort
Then once no unseen players are there, store as CSV file

Factor out everything into a new repo

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from base64 import b64encode as b64
import asyncio
import time
import ast
import itertools


In [2]:
# from username_to_user_id import username_to_user_id # from original repo
# old_names = set(username_to_user_id.keys())
# print("old names:  ", len(old_names))

# ratings_url = "https://acquire.tlstyer.com/stats/data/ratings.json"
# resp = requests.get(ratings_url)
# stats = json.loads(resp.text)

# rated_names = set()
# for key in stats.keys():
#     for rating in stats[key]:
#         name = rating[0]
#         rated_names.add(name)
# print("rated names:  ", len(rated_names))


# player_page_root = "https://acquire.tlstyer.com/stats/data/users/" # /encoded_username.json
# # extract all names from JSON response

In [3]:
# def encode_username(s_ascii = "username"):
#     s_code = b64(s_ascii.encode("utf-8")).decode("utf-8")
#     s_code = s_code.replace("=", "")
#     s_code = s_code.replace("+", "")
#     s_code = s_code.replace("\/", "_")
#     return s_code

In [4]:
# print(len(list(rated_names) + list(old_names)))
# print(len(rated_names.union(old_names)))
# print(len(rated_names.intersection(old_names)))

In [5]:
# players = pd.DataFrame()
# usernames = pd.Series(list(rated_names.union(old_names)))
# players['username'] = usernames
# players['encoded_username'] = players['username'].apply(encode_username)
# players['url'] = player_page_root + players['encoded_username'] + ".json"
# players

In [6]:
# players['find_degree'] = np.nan

# flavors = ['S2', 'S3', 'S4', 'T']
# attrs = ['rating', 'rate_moe', 'games_count', 'record', 'last_played']

# words = list(itertools.product(flavors, attrs))
# cols = ["_".join(two) for two in words]

# for col in cols:
#     players[f'{col}'] = np.nan

# players.info()


In [7]:
# players.to_pickle("./players.pkl")

In [8]:
# players = pd.read_pickle("./players.pkl")
# players.info()

In [9]:
# len(players[players['stats_page'] == ""])

In [10]:
# players['stats_page'] = players['stats_page'].str.decode("utf-8")

In [11]:
# sum(players['stats_page'].isna())

In [12]:
# opponents = set()

In [13]:
# sum(players['username'] == "test1")  # known username from server file

In [14]:
# known_players = set(players['username'])
# new_players = set()

In [381]:
valid_users = players[~players['stats_page'].isna()]

#Extract ratings:
valid_user = valid_users.sample()
stats_object = json.loads(valid_user['stats_page'].values[0])
ratings = stats_object['ratings']
    # return ratings

games = stats_object['games']
print("")
print(len(games))

#Extract Games Opponents:
for game in games:
    match_player = (game[2][0][0])
    if match_player not in known_players:
        if match_player not in new_players:
            new_players.add(match_player)
    # return new_players
print(len(new_players))


11
96


In [382]:
print(new_players)

{'YumYum', 'hoff2021', 'Jkr4', 'Domkop', 'focus2021', 'xiaopiting', 'eero', 'baroquester', 'Thales College Student', 'zigzag', 'just play fast', 'zeevr-BTW', 'spookygirl101', 'Tom Wambsgans', 'Shaggy', 'Sid the Kid', 'aaron2021', 'disenchanted 66.237', 'gogogadget', 'Bruno', 'Hilary', 'Nurse', 'prettykitty', 'Tune', '2300skidoo', 'Ron DeSantis', 'space672', 'Cleveland Indians', 'cupcake', 'Juice Box', 'ahmed.hany.abdelwahab@gmail.com', 'trent025', 'train7878', 'kurtweis22', 'stewie', 'space21', 'gmoney', 'LeopardGeckoFan', 'Skibaba2121', 'nevermind', 'Frank', 'Used2bA', 'piruli22', 'Wolvo', 'LSU', 'janssentan96', 'alwayswinning', 'Buddy6985', 'businessman2021', 'billyboy34', 'empireofthemind', 'DrSolano', 'H-Rah2022', 'Kali', '##', 'Alias2021', 'A.A.Ron', 'Doc Holliday', 'Spaceeer', 'Jakob-dec', 'dwomo', 'Fahrver21', 'itsamegary', 'Cousin Greg', 'dist347', 'cosmocougar!', 'tbow', 'Ruffy', 'GaryTx', 'mrblac', '2021plsbbttr', 'IAdoug2021', 'magenta', 'sllubad2003', 'Crown13', 'muditb', '

In [14]:
players = pd.read_csv("./players.csv")

In [15]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23701 entries, 0 to 23700
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        23701 non-null  int64  
 1   username          23700 non-null  object 
 2   encoded_username  23700 non-null  object 
 3   url               23701 non-null  object 
 4   wave              23701 non-null  int64  
 5   ratings           0 non-null      float64
 6   S2_rating         0 non-null      float64
 7   S2_games_count    0 non-null      float64
 8   S2_record         0 non-null      float64
 9   S2_last_played    0 non-null      float64
 10  S3_rating         0 non-null      float64
 11  S3_games_count    0 non-null      float64
 12  S3_record         0 non-null      float64
 13  S3_last_played    0 non-null      float64
 14  S4_rating         0 non-null      float64
 15  S4_games_count    0 non-null      float64
 16  S4_record         0 non-null      float6

In [18]:
# stats_object = json.loads(valid_user['stats_page'].values[0])
# ratings = stats_object['ratings']
blank_pagers = players['stats_page'].isna()

In [19]:
players['ratings'] = players[~blank_pagers]['stats_page'].apply(json.loads)

In [24]:
blank_ratings = players['ratings'].isna()

In [60]:
players[~blank_ratings]['ratings'].apply(lambda x: print(x['ratings']) )

{'Singles4': [1549688947, 22.7044, 6.0192, [1, 0, 0, 1]]}
{'Singles3': [1593956063, 28.705, 6.40931, [1, 1, 0]], 'Singles4': [1593973809, 23.2097, 6.86943, [0, 0, 1, 0]]}
{'Singles3': [1571966578, 20.3294, 7.38299, [0, 0, 1]]}
{'Singles4': [1577924339, 27.4684, 6.54264, [0, 1, 0, 0]]}
{'Singles3': [1633886188, 22.0178, 1.32371, [26, 38, 33]], 'Singles4': [1610722608, 25.0084, 2.85991, [3, 5, 3, 3]]}
{'Singles3': [1543243230, 28.3813, 4.98296, [1, 3, 0]], 'Singles4': [1546280254, 23.5079, 0.955977, [184, 200, 216, 218]], 'Teams': [1525358599, 22.6192, 7.8818, [0, 1]]}
{'Singles3': [1598745935, 25.7085, 6.86603, [0, 1, 0]]}
{'Singles2': [1616183519, 12.7508, 6.39916, [0, 2]], 'Singles3': [1621094474, 23.9525, 6.98009, [0, 1, 0]], 'Singles4': [1656676651, 21.0283, 1.02352, [35, 49, 40, 62]], 'Teams': [1617543285, 17.5543, 7.02633, [0, 4]]}
{'Singles4': [1514620815, 11.6827, 5.02976, [0, 0, 1, 3]]}
{'Singles2': [1603083489, 20.2047, 4.13214, [3, 8]], 'Singles4': [1603031047, 20.3103, 3.511

0        None
2        None
3        None
7        None
10       None
         ... 
23701    None
23703    None
23704    None
23705    None
23706    None
Name: ratings, Length: 14029, dtype: object

In [61]:
def get_s2_rating(row):
    if 'Singles2' in row['ratings']:
        return row['ratings']['Singles2']
    return None

def get_s3_rating(row):
    if 'Singles3' in row['ratings']:
        return row['ratings']['Singles3']
    return None

def get_s4_rating(row):
    if 'Singles4' in row['ratings']:
        return row['ratings']['Singles4']
    return None

def get_t_rating(row):
    if 'Teams' in row['ratings']:
        return row['ratings']['Teams']
    return None

In [63]:
rates = players[~blank_ratings]['ratings'].apply(get_s4_rating)
rates.info()

<class 'pandas.core.series.Series'>
Int64Index: 14029 entries, 0 to 23706
Series name: ratings
Non-Null Count  Dtype 
--------------  ----- 
10536 non-null  object
dtypes: object(1)
memory usage: 219.2+ KB


In [56]:
for rate in rates:
    if rate:
        for item in rate:
            print(item)

1549688947
22.7044
6.0192
[1, 0, 0, 1]
1593973809
23.2097
6.86943
[0, 0, 1, 0]
1577924339
27.4684
6.54264
[0, 1, 0, 0]
1610722608
25.0084
2.85991
[3, 5, 3, 3]
1546280254
23.5079
0.955977
[184, 200, 216, 218]
1656676651
21.0283
1.02352
[35, 49, 40, 62]
1514620815
11.6827
5.02976
[0, 0, 1, 3]
1603031047
20.3103
3.51117
[1, 1, 3, 4]
1587065129
26.6433
6.82387
[0, 1, 0, 0]
1610921857
26.887
6.80494
[0, 1, 0, 0]
1421040823
28.413
5.47295
[2, 0, 0, 1]
1475732156
22.5646
6.60262
[0, 0, 1, 0]
1518367071
20.5149
1.74982
[6, 8, 10, 18]
1549841537
19.4509
7.04935
[0, 0, 0, 1]
1584865193
15.4739
6.46739
[0, 0, 0, 2]
1521778822
15.8263
4.02235
[1, 0, 1, 5]
1673240643
16.8123
4.92659
[0, 1, 0, 3]
1483719609
16.0117
5.01257
[0, 0, 1, 3]
1512712936
18.0916
7.04275
[0, 0, 0, 1]
1579912182
24.1578
6.53677
[0, 0, 1, 0]
1478067994
23.7977
6.52939
[0, 0, 1, 0]
1594962326
27.1095
6.76078
[0, 1, 0, 0]
1465513964
19.2619
4.34319
[0, 1, 2, 2]
1605999728
27.7167
4.04529
[3, 1, 2, 1]
1590717301
22.3079
6.49051
[

16.0315
4.78057
[1, 0, 0, 4]
1585191503
23.5385
6.1347
[1, 0, 0, 1]
1481516328
19.2097
5.88228
[0, 0, 1, 1]
1596972552
18.8407
6.98484
[0, 0, 0, 1]
1485825493
34.3604
4.6986
[3, 2, 0, 0]
1546322033
29.3975
2.00525
[12, 8, 5, 5]
1599885036
21.5777
6.50437
[0, 0, 1, 0]
1565744732
14.1671
3.24986
[1, 1, 1, 10]
1586924457
23.9753
6.86405
[0, 0, 1, 0]
1579393769
19.0457
2.58764
[1, 3, 6, 8]
1576775530
23.8861
6.50093
[0, 0, 1, 0]
1551318069
25.6252
6.68065
[0, 1, 0, 0]
1594150930
14.285
6.14194
[0, 0, 0, 2]
1640401219
21.9214
1.00753
[36, 38, 62, 81]
1673467573
22.0601
0.964918
[1138, 1225, 1338, 1463]
1622584130
25.0085
4.63853
[1, 1, 1, 1]
1673560173
26.9739
0.955829
[908, 821, 735, 580]
1527970215
23.1665
2.60501
[4, 1, 7, 5]
1582175543
18.8218
7.1824
[0, 0, 0, 1]
1597873502
15.5993
2.90875
[0, 1, 8, 5]
1520183557
17.6846
2.21496
[2, 4, 7, 14]
1663522612
31.1718
7.00887
[1, 0, 0, 0]
1600262323
19.5975
7.05197
[0, 0, 0, 1]
1539643235
19.4537
5.8231
[0, 0, 1, 1]
1477321429
24.3773
6.50496


In [64]:
players.columns

Index(['Unnamed: 0', 'username', 'encoded_username', 'url', 'wave', 'ratings',
       'S2_rating', 'S2_rate_moe', 'S2_games_count', 'S2_record',
       'S2_last_played', 'S3_rating', 'S3_rate_moe', 'S3_games_count',
       'S3_record', 'S3_last_played', 'S4_rating', 'S4_rate_moe',
       'S4_games_count', 'S4_record', 'S4_last_played', 'T_rating',
       'T_rate_moe', 'T_games_count', 'T_record', 'T_last_played',
       'stats_page'],
      dtype='object')

In [ ]:
players['']

In [51]:
shorties = players['username'].str.len() > 30
sum(shorties)

87